In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import time
from pprint import pprint
import pandas as pd
from selenium.webdriver.chrome.options import Options
import requests
import json
from config import geoapify_key
import re
import numpy as np


browser = Browser('chrome')
browser.driver.maximize_window()

In [2]:

# Load the web page
url = "https://www.lcbo.com/en/products#t=clp-products&sort=relevancy&layout=card&f:@lcbo_current_offer=[On%20Sale]&f:@ec_category=[Wine]"
browser.visit(url)

# To select "English" as preffered language
time.sleep(3)
results = browser.links.find_by_text('English')
results.click()
time.sleep(3)

html = browser.html
soup=BeautifulSoup(html, 'html.parser')

ElementDoesNotExist: no elements could be found with link by text "English"

In [ ]:
# Removes the cookie popup
cookie = browser.find_by_id('btn-cookie-allow').click()

In [ ]:
iteration = 0
while True:
    try:     
        # Wait for 1 second
        time.sleep(2)
        # Click on Load More button
        browser.find_by_id('loadMore').click()
    except:
        break
    print(iteration)
    iteration += 1

In [ ]:
html = browser.html
    
soup = BeautifulSoup(html, 'html.parser')

all_links = soup.find_all("div", class_ = 'coveo-product-items')

links = []

for link in all_links:
    results = link.a['href']
    links.append(results)        

In [ ]:
print(len(links))


In [ ]:
# div_element = soup.find('div', class_='fotorama__stage__frame')
# if div_element and 'href' in div_element.attrs:
#     link = div_element['href']
#     print(link)
# else:
#     print("Link not found.")

In [ ]:
regular_price = []
sale_price = []
name = []
rating = []
onsale = [] #this is the On Sale tag
reviews = []
size = []
description = []
category = []
details_list = []
taste_list = []
images = []


max_iteration2 = len(links)
iteration2 = 0


for record in links:
    if iteration2 >= max_iteration2:
        break
    url = record
    browser.visit(url)
    time.sleep(2)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
#     time.sleep(1)
#     browser.find_by_css('my_store_close_button').click()

    try:
        div_element = soup.find('div', class_='fotorama__stage__frame')
        if div_element and 'href' in div_element.attrs:
            link = div_element['href']
            images.append(link)
        else:
            print("Link not found.")
    except AttributeError:
        images.append('N/A')
        
    try:
        old_price = soup.find('span', class_ = 'old-price')
        old = old_price.find('span', class_ = 'price')
        regular_price.append(old.find('span', class_ = 'price').text)
    except AttributeError:
        regular_price.append('N/A')

    try:
        on_sale_price = soup.find('span', class_ = 'special-price')
        new_price = on_sale_price.find('span', class_ = 'price')
        sale_price.append(new_price.find('span', class_ = 'price').text)
    except AttributeError:
        sale_price.append('N/A')
    
    try:    
        name.append(soup.find('div', class_ = 'page-title-wrapper').text)
    except AttributeError:
        name.append('N/A')
        
    try:
        onsale.append(soup.find('div', class_ = 'amlabel-text').text)
    except AttributeError:
        onsale.append('N/A')
    
    try:
        rating.append(soup.find('div', class_ = 'bv_avgRating_component_container notranslate').text)
    except AttributeError:
        rating.append('N/A')
    
    try:
        reviews.append(soup.find('div', class_ = 'bv_numReviews_text').text)
    except AttributeError:
        reviews.append('N/A')
    
    try:
        size.append(soup.find('div', class_ = 'lcbo-product-size').text)
    except AttributeError:
        size.append('N/A')
        
    try:
        description.append(soup.find('div', class_ = 'testing_note').text)
    except AttributeError:
        description.append('N/A')
        
    categories_all = soup.find('div', class_ = 'breadcrumbs')
    categories = categories_all.find_all('li', class_ = 'item')
    category_list = []

    for category_record in categories:
        category_list.append(category_record.text)
    category.append(category_list[-2])

    moredetail = soup.find('div', class_ = 'moredetail')
    all_details = moredetail.find_all('li')
    details = {}

    for detail_record in all_details:
        details_value = detail_record.find('div', class_ = 'value').text
        details_name = detail_record.find('div', class_ = 'label').text
        details[details_name] = details_value

    details_list.append(details)
    
#     try:
#         flavour_all = soup.find('div', class_ = 'foodParings pip-info')
#         flavour = flavour_all.find('div', class_ = 'label').text
#         flavour_value = flavour_all.find('div', class_ = 'value').text
#         flavours = {
#             flavour: flavour_value
#         }
#     except AttributeError:
#         flavours = {
#             'Flavour': 'N/A'
#         }
        
    
#     try:
#         all_taste = soup.find('div', class_ = 'foodParings pip-info')
#         li_all = all_taste.find_all('li')

#         for li in li_all:
#             label = li.find('div', class_='label').text.strip()
#             value_element = li.find('div', class_='value')
#             if 'aria-label' in value_element.attrs:
#                 aria_label = value_element['aria-label']
#                 number = re.findall(r'\d+', aria_label)
#                 if number:
#                     taste_list.append({
#                         label: int(number[0])
#                     })
#     except AttributeError:
#         taste_list.append('N/A')
        
    print(iteration2)
    iteration2 += 1



In [ ]:
data = {
    'Name': name,
    'Regular Price': regular_price,
    'Sale Price': sale_price,
    'Rating': rating,
    'Reviews': reviews,
    'Size': size,
    'Description': description,
    'Category': category,
    'Details': details_list,
    'Image': images
#     'Flavours': flavours
    
}

# Add the details dictionary to the data dictionary
data['Details'] = details_list
# data['Taste'] = taste_list

# Create a dataframe from the data dictionary
df = pd.DataFrame(data)

details_df = pd.json_normalize(df['Details'])
# taste_df = pd.json_normalize(df['Taste'])

df = pd.concat([df, details_df], axis = 1)
# df2 = pd.concat([df1, taste_df], axis = 1)

df.drop('Details', axis = 1, inplace = True)
# df2.drop('Taste', axis = 1, inplace = True)

#Adding Latitude and Longitude columns to the Data Frame
df['Lat'] = ""
df['Lon'] = ""   

In [ ]:
print(len(regular_price))
print(len(sale_price))
print(len(name))
print(len(rating))
print(len(onsale))
print(len(reviews))
print(len(size))
print(len(description))
print(len(category))
print(len(details_list))
print(len(images))
# print(len(flavours))
# print(flavours)

In [ ]:
#API call to get Latitude and Longitude
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

df['Lat'] = None
df['Lon'] = None

for index, country in df['Made In'].to_frame().iterrows():
    location = country['Made In']
    params["text"] = location
    
    response = requests.get(base_url, params=params).json()

    latitude = response["results"][0]["lat"]
    longitude = response["results"][0]["lon"]
    df.loc[index, "Lat"] = latitude
    df.loc[index, "Lon"] = longitude

df.head()


In [ ]:
df.loc[df['Name'] == ' Allegrini Palazzo della Torre 2019', 'Regular Price']

In [ ]:
df['Regular Price'] = df['Regular Price'].str.replace('$','')
df['Sale Price'] = df['Sale Price'].str.replace('$','')


df['Savings'] = df['Regular Price'].astype(float) - df['Sale Price'].astype(float)

df['Alcohol/Vol'] = df['Alcohol/Vol'].str.rstrip('%')
df['Alcohol/Vol'] = pd.to_numeric(df['Alcohol/Vol']) /100
df['Image'] = df['Image'].str.replace('1280.1280','319.319')

df.head()

In [ ]:
df.rename(columns={'Sugar Content' : 'Sugar Content g/L'})

In [ ]:
nan_df = df.isna()
df = df.fillna('N/A')
df.head()

In [ ]:
column_lists = {column: df[column].tolist() if not np.issubdtype(df[column].dtype, np.number) else df[column].fillna('N/A').tolist() for column in df.columns}

name_list = column_lists['Name']
regular_price_list = column_lists['Regular Price']
sale_price_list = column_lists['Sale Price']
rating_list = column_lists['Rating']
reviews_list = column_lists['Reviews']
size_list = column_lists['Size']
description_list = column_lists['Description']
category_list = column_lists['Category']
alcohol_list = column_lists['Alcohol/Vol']
madein_list = column_lists['Made In']
by_list = column_lists['By']
varietal_list = column_lists['Varietal']
lat_list = column_lists['Lat']
lon_list = column_lists['Lon']
savings_list = column_lists['Savings']
images_list = column_lists['Image']

lcbo_1 = {
    'name': name_list,
    'regular_price': regular_price_list,
    'sale_price': sale_price_list,
    'rating': rating_list,
    'reviews': reviews_list,
    'size': size_list,
    'description': description_list,
    'category': category_list,
    'alcohol_vol': alcohol_list,
    'made_in': madein_list,
    'by': by_list,
    'varietal': varietal_list,
    'lat': lat_list,
    'lon': lon_list,
    'savings': savings_list,
    'image': images_list
}

with open('lcbo_1.json', 'w') as file:
    json.dump(lcbo_1, file)

In [ ]:
column_lists = {column: df[column].tolist() if not np.issubdtype(df[column].dtype, np.number) else df[column].fillna('N/A').tolist() for column in df.columns}

name_list = column_lists['Name']
regular_price_list = column_lists['Regular Price']
sale_price_list = column_lists['Sale Price']
rating_list = column_lists['Rating']
reviews_list = column_lists['Reviews']
size_list = column_lists['Size']
description_list = column_lists['Description']
category_list = column_lists['Category']
alcohol_list = column_lists['Alcohol/Vol']
madein_list = column_lists['Made In']
by_list = column_lists['By']
varietal_list = column_lists['Varietal']
lat_list = column_lists['Lat']
lon_list = column_lists['Lon']
savings_list = column_lists['Savings']
images_list = column_lists['Image']

lcbo_2 = {
    'name': name_list,
    'regular_price': regular_price_list,
    'sale_price': sale_price_list,
    'rating': rating_list,
    'reviews': reviews_list,
    'size': size_list,
    'description': description_list,
    'category': category_list,
    'alcohol_vol': alcohol_list,
    'made_in': madein_list,
    'by': by_list,
    'varietal': varietal_list,
    'lat': lat_list,
    'lon': lon_list,
    'savings': savings_list,
    'image': images_list
}

with open('lcbo_2.json', 'w') as file:
    file.write(pd.DataFrame(lcbo_2).to_json(orient='records'))

In [ ]:
with open('lcbo_1.json', 'r') as file1:
    lcbo1 = json.load(file1)

# Read the contents of the second JSON file
with open('lcbo_2.json', 'r') as file2:
    lcbo2 = json.load(file2)

# Merge the two JSON data

merged_data = lcbo2
merged_data.extend(lcbo1)

# Write the merged data to a new JSON file
with open('lcbo_merged.json', 'w') as outfile:
    json.dump(merged_data, outfile)

In [ ]:
df.to_csv('lcbo_wines.csv', index = False)